## 서브쿼리란?
- 다른 SELECT문에 삽입된 SELECT문 입니다.
- 테이블 자체 데이터에 종속된 조건을 사용하여 테이블에서 행을 선택할 때 유용
- 사용지침
    - 괄호()로 묶어야함
    - 비교조건의 오른쪽에 서브쿼리를 넣음
    - ORDER BY절을 사용하지 않음
    - 단일행 연산자(>,<,=,<>등)와 복수행 연산자(IN,ANY,ALL)를 사용

---

## WHERE절 서브쿼리
- WHERE절에 사용된 서브쿼리 실행순서는 서브쿼리를 먼저 수행하고 그 결과값을 이용해 메인 쿼리 조건을 비교

서동명 선수와 같은 팀 선수들 이름, 팀id, 포지션 정보를 조회

<pre>
SELECT player_name, team_id, position
FROM player
WHERE team_id = ( SELECT team_id
                  FROM player
                  WHERE player_name = '서동명' );
</pre>

전체 선수들 평균키 보다 키가 큰 선수들만 선수id, 이름, 키, 팀id 정보를 조회

<pre>
SELECT player_id, player_name, height ,team_id
FROM player
WHERE height > ( SELECT avg(height) 
                 FROM player );
</pre>

포지션은 서동명 선수와 같고 키는 디디 선수보다 큰 선수들만 이름, 포지션 ,키 조회

<pre>
SELECT player_name, position, height
FROM player
WHERE position = ( SELECT position 
                   FROM player
                   WHERE player_name='서동명' )
AND height > ( SELECT height
               FROM player
               WHERE player_name='디디' );
</pre>

---

## 다중행 서브쿼리
- 서브쿼리 결과가 여러행을 반환하는 경우 다중 행 서브쿼리

    - IN : 서브쿼리 결과 목록에 있는 임의의 값과 같은지 비교
    
    
    - ANY : 같은 반환되는 각각의 값과 개별 비교해서 조건을 만족하는지 확인
        - '< ANY' : 최대값보다 적음을 의미
        - '> ANY' : 최소값보다 큼을 의미
        - '= ANY' : IN과 동일하게 처리
        
        
    - ALL : 서브쿼리에 의해 반환되는 모든 값과 비교
        - '< ALL' : 최소값보다 적음을 의미
        - '> ALL' : 최대값보다 큼을 의미

player 테이블 K02팀에 소속된 모든 선수들보다 키가 큰 선수들의 이름, 키를 조회

<pre>
SELECT player_name, height
FROM player
WHERE height > ALL( SELECT height
                    FROM player
                    WHERE team_id = 'K02'); 
</pre>
<br>

ALL 키워드대신 최대키값과 비교한 결과를 확인

<pre>
SELECT player_name, height
FROM player
WHERE height > ( SELECT MAX(height)
                 FROM player
                 WHERE team_id = 'K02'); 
</pre>

player 테이블 K02팀에 소속된 최소 한명의 선수보다 키가 큰 선수들이 있으면 이름, 키를 조회

<pre>
SELECT player_name, height
FROM player
WHERE height > ANY ( SELECT height
                     FROM player
                     WHERE team_id = 'K02'); 
</pre>
<br>


ANY 키워드대신 최소키값과 비교한 결과를 확인
<pre>
SELECT player_name, height
FROM player
WHERE height > ( SELECT MIN(height)
                 FROM player
                 WHERE team_id = 'K02');
</pre>                 
             

---

## FROM절 서브쿼리
- 서브쿼리가 FROM절에 명시되면 새로운 데이터 소스를 생성
- FROM절에 기술한 서브쿼리는 마치 뷰를 생성하고 뷰를 SELECT문에서 부르는 것과 개념상 비슷
- 데이터 소스처럼 사용되는 서브쿼리는 저장되지 않음
- SELECT문 일부로서 일시적으로만 존재
- 인라인 뷰라고도 함

team, player이용 팀명, 연고지명, 해당팀 선수들 평균키를 조회

<pre>
SELECT t.team_name, t.region_name, avg(p.height) 
FROM player p 
JOIN team t 
ON p.team_id = t.team_id 
GROUP BY t.team_name, t.region_name;
</pre>
- player테이블 모든 자료를 team테이블과 조인한 다음 팀명 , 연고지로  GROUP BY 작업을 수행
<br>
---


서브쿼리를 FROM절에 사용하여 조회
<pre>
SELECT t.team_name, t.region_name, p.avg_h
FROM    
(
  	SELECT team_id, avg(height) avg_h
    FROM player
    GROUP BY team_id 
) p 
JOIN team t 
ON p.team_id = t.team_id;
</pre>
- player테이블 자료를 팀id기준 GROUP BY작업을 먼저 수행후 team테이블과 조인 작업을 수행